# Simulus Basics

This guide describes the basics of how simulus works. All the examples shown in this guide can be found under the `examples/basics` directory in the simulus source-code distribution.

Simulus is a process-oriented discrete-event simulator written in Python. That's quite a mouthful. Let's first break it down what it means. 

## Discrete-Event Simulation and Process-Oriented World View

A discrete-event simulator models the world as a sequence of events that happen in discrete time. The simulator basically processes these events in order. Such a simulator would keep a simulation clock, which advances its time by leaps and bounds in accordance with the time of the events being processed along the way.

For example, a professor starts the day with a cup of coffee at 4 AM, reads for 2 hours, has breakfast, takes shower, and then drives to school at 7:30 AM. The discrete-event simulator for the professor's day will go through a sequence of events: "wake up" event at 4:00, "start coffee" event at 4:05, "start read" event at 4:10, "finish coffee" event at 4:20, "finish read" event at 6:10, breakfast event at 6:30, shower event at 6:50, "start driving" event at 7:30, etc. The clock of the simulator in this case will go through a sequence of distinct values: 4:00, 4:05, 4:10, ... You got the gist. 

Discrete-event simulation is a very powerful modeling method. You can think of the entire world, including the activities of all professors, students, and all people, as a sequence of events that happen over time. The simulator's job is to play out the events in order. By doing so, we get better understanding of the world.

Simulus is a "process-oriented" discrete-event simulator. What it means is that in addition to describing the world as a bunch of events, one can also model it using processes. As an example, suppose the world consists of many professors, students, and people. Previously we described the schedule of one professor. A different person would have a different schedule, i.e., following a different sequence of events. Using process-oriented simulation, every person in this system can be modeled as a separate process. 

Note that people interact in this world. For example, if everyone starts driving to school or work in the morning at the same hour, most people would get delayed since congestion would happen on the road. Also, if a student needs to meet with another student at a coffee shop. They would not be able to carry on with their next task (say, to work on their joint project) until both of them can get to the coffee shop. Simulus provides the needed support for creating and managing the processes, making it easier for one to model the world.

## How Simulus Works

Simulus works in two ways. One way is through events. You can schedule events and process them. We call it *direct event scheduling*. The other way is through processes. You can create processes and have them run and interact. We call it *process scheduling*. Of course, there's a third way, by combining them and having them both.

### Direct Event Scheduling

In simulus, an event is simply a function to be invoked at a designated time. This can be illustrated using the following hello-world example.

#### The Hello-World Example

In [1]:
# examples/basics/helloworld.py
import simulus

def print_message(sim, params):
    print("Hello world at time "+str(sim.now))
    
sim = simulus.simulator()
sim.sched(print_message, until=10)
sim.run()

Hello world at time 10


In order to use the simulator, we need to first import the `simulus` module and then create a simulator using the `simulator()` method. A simulator maintains an event list where all events will be stored and sorted in their timestamp order. A simulator also keeps the time, which gets advanced while executing the events. One can find out the current time of the simulator by inspecting the `now` variable of the simulator.

In the example, the function `print_message()` is scheduled to run at time 10. We schedule the function using the `sched()` method of the simulator, and passing the name of the function and using the `until` argument to specify the time at which the function should be invoked.

In discrete-event simulation terminology, the function is also called an *event handler*. An event handler in simulus must take two arguments. The first argument `sim` is the simulator on which the function is scheduled to run. The second argument `params` is a dictionary used for passing all user-defined arguments, which we will describe momentarily.

To run the simulation, simply use the `run()` method of the simulator. Without an argument, the `run()` method will process *all* events on the event list and returns when there's no more left.

#### Passing Arguments to Event Handlers

The next example shows how to pass arguments between `sched()` and the event handler. 

In [2]:
# examples/basics/passargs.py
import simulus

def print_params(s, x):
    print("print_params() invoked at time "+str(s.now))
    print("  with msg:", x.get('msg', "hey, how can you forget the message?"))
    print("  all arguments:", x)
    
sim = simulus.simulator()
sim.sched(print_params, until=10, msg="hello", var=False)
sim.sched(print_params, until=20, arg1="here", params={"arg1":10, "arg2":"be good"})
sim.run()


print_params() invoked at time 10
  with msg: hello
  all arguments: {'msg': 'hello', 'var': False}
print_params() invoked at time 20
  with msg: hey, how can you forget the message?
  all arguments: {'arg1': 'here', 'arg2': 'be good'}


In this example, the `print_params()` function is scheduled to run both at time 10 and time 20. We can pass aguments from `sched()` to the event handler, either as keyword arguments (such as `msg`, `var`, and `arg1`), or in a dictionary argument explicitly named `params`, or both. They will also be turned into a dictionary and passed in as the second argument to the event handler when it is invoked. Note that in this example, when `arg1` appears both as a keyword argument and in the dictionary given to `params`, the keyword argument takes precedence. 

Of course, one can schedule more functions to run within an event handler. The whole point of simulation is to help capture and examine the complicated logic or processes of the world (or system) as the events unfold. 

#### The Life of a Professor

In the next example, we show the "complicated" life of a professor.

In [3]:
# examples/basics/professor.py
import simulus

from time import gmtime, strftime
def nowstr(sim):
    return strftime("%H:%M:%S", gmtime(sim.now))

def wake_up(sim, params):
    print("professor wakes up at "+nowstr(sim))
    sim.sched(start_coffee, offset=5*60) # 5 minutes from now
    sim.sched(breakfast, offset=2*3600+30*60) # 2 hours and 30 minutes from now
    sim.sched(shower, offset=2*3600+50*60) # 2 hours and 50 minutes from now
    sim.sched(leave, offset=3*3600+30*60) # 3 hours and 30 minutes from now
    
def start_coffee(sim, params):
    print("professor starts drinking coffee at "+nowstr(sim))
    sim.sched(finish_coffee, offset=15*60) # 15 minutes from now
    sim.sched(start_read, offset=5*60) # 5 minutes from now

def finish_coffee(sim, params):
    print("professor finishes drinking coffee at "+nowstr(sim))
    
def start_read(sim, params):
    print("professor starts reading at "+nowstr(sim))
    sim.sched(finish_read, offset=2*3600) # 2 hours from now
    
def finish_read(sim, params):
    print("professor finishes reading at "+nowstr(sim))

def breakfast(sim, params):
    print("professor breakfasts at "+nowstr(sim))

def shower(sim, params):
    print("professor shows at "+nowstr(sim))

def leave(sim, params):
    print("professor leaves home and drives to school at "+nowstr(sim))
    sim.sched(arrive, offset=45*60) # 45 minutes from now

def arrive(sim, params):
    print("professor arrives at school at "+nowstr(sim))

def meeting1(sim, params):
    print("professor has first meeting at "+nowstr(sim))

def meeting2(sim, params):
    print("professor has second meeting at "+nowstr(sim))

sim = simulus.simulator()
sim.sched(wake_up, until=4*3600) # 4:00
sim.sched(meeting1, until=9*3600) # 9:00
sim.sched(meeting2, until=10*3600) # 10:00
sim.run()

professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor shows at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor has first meeting at 09:00:00
professor has second meeting at 10:00:00


The time is represented as a floating point number in simulus. In this example, we use time in seconds starting from  midnight. To make things a bit more comprehensible, we use the standard `time` module in Python to turn the seconds into hours and minutes in the printout. 

#### Canceling and Rescheduling Events

So far, the professor's schedule is rather boring. In real life, we make appointments and we cancel appointments. We also reschedule appointments. Let's make things a bit more interesting for the professor's life. Let's assume that today there's a big traffic jam. So instead of taking only 45 minutes, the professor takes 2 hours 45 minutes to get to school. Obviously the professor will miss the first meeting and get late for the second meeting. So she cancels the first meeting and reschedules the second meeting. 

To do that we need to use the return value from the `sched()` method. It is actually a reference to an event created by the sched() method. Event in simulus is an opaque object. That is, we can use the event reference to cancel or reschedule the event, but we should not directly access the variables and methods within the event class. In this example, we call the simulator's `cancel()` and `resched()` methods and pass the event as the argument. The professor cancels and reschedules the events when she is about to leave home (assuming she predicts that the traffic jam is about to happen).

Another difference of this example from the previous one is that we make professor's waking up a daily repeatable event (with a 24 hour interval). In this case, the professor always gets up at 4 AM every day and carries out the same set of activities. If we `run()` the simulation without an argument, it would never return as there will always be events on the event list. Because of that, in this example, we call `run()` with a simulation end time using the named argument `until`. We run simulation for a duration of 3 days.

In [4]:
# examples/basics/professor-flex.py
import simulus

from time import gmtime, strftime
def nowstr(sim):
    return strftime("%H:%M:%S", gmtime(sim.now))

def wake_up(sim, params):
    print("professor wakes up at "+nowstr(sim))
    sim.sched(start_coffee, offset=5*60) # 5 minutes from now
    sim.sched(breakfast, offset=2*3600+30*60) # 2 hours and 30 minutes from now
    sim.sched(shower, offset=2*3600+50*60) # 2 hours and 50 minutes from now
    sim.sched(leave, offset=3*3600+30*60) # 3 hours and 30 minutes from now
    
def start_coffee(sim, params):
    print("professor starts drinking coffee at "+nowstr(sim))
    sim.sched(finish_coffee, offset=15*60) # 15 minutes from now
    sim.sched(start_read, offset=5*60) # 5 minutes from now

def finish_coffee(sim, params):
    print("professor finishes drinking coffee at "+nowstr(sim))
    
def start_read(sim, params):
    print("professor starts reading at "+nowstr(sim))
    sim.sched(finish_read, offset=2*3600) # 2 hours from now
    
def finish_read(sim, params):
    print("professor finishes reading at "+nowstr(sim))

def breakfast(sim, params):
    print("professor breakfasts at "+nowstr(sim))

def shower(sim, params):
    print("professor showers at "+nowstr(sim))

def leave(sim, params):
    print("professor leaves home and drives to school at "+nowstr(sim))
    if sim.now < 24*3600:
        # traffic jam at the first day
        sim.sched(arrive, offset=2*3600+45*60) # 2 hours and 45 minutes from now
        # the two meetings are only at the first day
        sim.cancel(e1)
        sim.resched(e2, until=11*3600) # 11:00
    else:
        # no traffic jam in the following days
        sim.sched(arrive, offset=45*60) # 45 minutes from now

def arrive(sim, params):
    print("professor arrives at school at "+nowstr(sim))

def meeting1(sim, params):
    print("professor has first meeting at "+nowstr(sim))

def meeting2(sim, params):
    print("professor has second meeting at "+nowstr(sim))

sim = simulus.simulator()
sim.sched(wake_up, until=4*3600, repeat_intv=24*3600) # 4:00 
e1 = sim.sched(meeting1, until=9*3600) # 9:00
e2 = sim.sched(meeting2, until=10*3600) # 10:00
sim.run(until=72*3600)

professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 10:15:00
professor has second meeting at 11:00:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor sho

#### More on Running Simulators

By default, a simulator starts at time zero. Instead, we can use the `init_time` argument to change the start simulation time when we create the simulator. 

A simulator can also have an optional name using the `name` argument. If specified, the simulator's name should be unique. In this case, one can use the `get_simulator()` function to retrieve the simulator using the name. If the name is not unique, a simulator created later with the same name may overwrite the earlier one, which will no longer be retrievable using the name. A simulator's name is optional. A simulator can be created without a name. In this case, one cannot use the `get_simulator()` function to retrieve it.

All the previous examples create only one simulator at a time. In fact, one can simultaneously run multiple simulators in simulus, each maintaining its own event list and its own simulation time. If multiple simulators are created, they will all run independently. The event handlers at different simulators are invoked in separate timelines. That is, the current time at one simulator has no relation to the current time of another simulator. This is the default behavior. Simulators can also be time synchronized (using the `sync()` function). We will return to this topic in the advanced tutorial when we discuss the parallel and distributed simulation support of simulus.

Each simulator processes its own events when we call the `run()` method. We can specify either `offset` or `until` (but not both) when we call `run()`, in which case the simulator will process all events with timestamps no larger than the designated time. The `offset` argument specifies a relative time from the current time. The `until` argument specifies the absolute time. When the method returns, the current time of the simulator will be advanced to the designated time. If both `offset` and `until` are ignored, the simulator will run as long as there are events on the event list. Be careful in this case, a simulator may run forever for some models as new events are generated repeatedly when the event handlers are invoked (like in the previous example).

We can also step through simulation processing one event at a time. This is achieved using the simulator's `step()` method, which processes the next event on the event list and advances the simulation clock to the time of the event. If there is no event available on the event list, the method has no effect. 

To determine the time of the next event, we can use the `peek()` method of the simulator. We can also list all future events of a simulator using the `show_calendar()` method. This method is supposedly used only for debugging purposes. Listing all events on an event list can be an expensive operation.

The following example illustrates some of the functions we just metioned in this section.

In [5]:
# examples/basics/twosims.py
import simulus

def handle(sim, params):
    print("'%s' handles event at time %g" % (sim.name, sim.now))

sim1 = simulus.simulator(name="sim1", init_time=100)
sim2 = simulus.simulator(name="sim2", init_time=-100)

for i in range(5, 100, 20):
    sim1.sched(handle, offset=i) # use offset here
for i in range(5, 200, 10):
    sim2.sched(handle, offset=i) # use offset here
sim1.show_calendar()
sim2.show_calendar()

while True:
    t1, t2 = sim1.peek(), sim2.peek()
    if t1 < simulus.infinite_time:
        sim1.step()
    if t2 < simulus.infinite_time:
        sim2.step()
    if t1 == simulus.infinite_time and \
       t2 == simulus.infinite_time:
        break

list of future events (num=5) at time 100 on simulator sim1:
  DirectEvent[4409140280]:105
  DirectEvent[4409141568]:125
  DirectEvent[4409141736]:145
  DirectEvent[4409140448]:165
  DirectEvent[4409140000]:185
list of future events (num=20) at time -100 on simulator sim2:
  DirectEvent[4409142128]:-95
  DirectEvent[4409142296]:-85
  DirectEvent[4409142184]:-75
  DirectEvent[4409141120]:-65
  DirectEvent[4409140672]:-55
  DirectEvent[4409667936]:-45
  DirectEvent[4409668216]:-35
  DirectEvent[4409668272]:-25
  DirectEvent[4409667992]:-15
  DirectEvent[4409667656]:-5
  DirectEvent[4409667824]:5
  DirectEvent[4409668440]:15
  DirectEvent[4409668720]:25
  DirectEvent[4409668944]:35
  DirectEvent[4409669000]:45
  DirectEvent[4409668776]:55
  DirectEvent[4409668552]:65
  DirectEvent[4409669112]:75
  DirectEvent[4409669392]:85
  DirectEvent[4409669616]:95
'sim1' handles event at time 105
'sim2' handles event at time -95
'sim1' handles event at time 125
'sim2' handles event at time -85
'sim1'

In this example, we create two simulators, one starting from time 100 and the other from time -100. (Yes, we can use negative simulation time!) For one simulator, we schedule a function to be invoked 5 times at regular time intervals with a gap of 20 and starting at offset 5 from the simulator's start time. For the other simulator, we schedule the same function to be invoked 10 times at regular time intervals with a gap of 10 and starting at offset 5 from the simulator's start time. We then step through the scheduled events of the two simulators, processing one event at a time for each simulator in an alterative fashion as long as they have events.

### Process Scheduling

Another way to model the world is to use simulation processes. Conceptually, a process is just a thread of control, which is similar to sequentially running through a prgram, that constitutes a bunch of statements, including if-branches, while-loops, and function calls. During its execution, a simulation process can be blocked, either sleeping for some time, or requesting for some resources that are not currently available. The process will resume execution when the specified time has passed or after the resource blocking condition has been removed. 

Simulus implement the simulation processes as coroutines (also known as micro-threads or "tasklets"), using Python's greenlet package.

#### The Hello-World Example Using Process

The following is a modified hello-world example, which uses a process.

In [6]:
# examples/basics/helloworld-proc.py
import simulus

def print_message(sim, params):
    for _ in range(10):
        print("Hello world at time "+str(sim.now))
        sim.sleep(1)
    
sim = simulus.simulator()
sim.process(print_message, until=10)
sim.run()

Hello world at time 10
Hello world at time 11
Hello world at time 12
Hello world at time 13
Hello world at time 14
Hello world at time 15
Hello world at time 16
Hello world at time 17
Hello world at time 18
Hello world at time 19


In this example, the function print_message() is the starting function of a process. We create the process using the `process()` method of the simulator and schedule the process to run at time 10 using the `until` argument. 

The format of the starting function of a process is exactly the same as an event handler.  The function must take two arguments: the first argument is the simulator on which the process is scheduled to run, and the second argument is a dictionary that contains all user arguments. 

As in the earlier example using the `sched()` method, we can pass aguments to the starting function when we create the process, either as keyword arguments, or in a dictionary argument explicitly named `params`, or both. Both will be turned into a dictionary and passed as the second argument to the starting function when the process starts running.

In this modified helloworld example, the process loops for 10 times; at each iteration, it prints out a hello message and sleeps for 1 second. The `sleep()` method takes one argument: either an `offset` argument, which gives the amount of time the process will be put to sleep, or an `until` argument, which indicates the time at which the process should resume execution. We don't need to explicitly name the argument as `offset` as in this example; it's the default position argument.

#### The Professor's Life as a Process

Now we return to the previous example and use a process to simulate the professor's complicated life.

In [7]:
# examples/basics/professor-proc.py
import simulus

from time import gmtime, strftime
def nowstr(sim):
    return strftime("%H:%M:%S", gmtime(sim.now))

def prof_life(sim, params):
    while True:
        start_of_the_day = sim.now
        
        sim.sleep(4*3600) # 4 hours from midnight
        print("professor wakes up at "+nowstr(sim))
        
        sim.sleep(offset=5*60) # 5 minutes from now
        print("professor starts drinking coffee at "+nowstr(sim))
    
        sim.sleep(offset=5*60) # 5 minutes from now
        print("professor starts reading at "+nowstr(sim))

        sim.sleep(offset=(15-5)*60) # 15 minus 5 minutes from now
        print("professor finishes drinking coffee at "+nowstr(sim))

        sim.sleep(offset=2*3600-10*60) # 2 hours minus 10 minutes from now
        print("professor finishes reading at "+nowstr(sim))

        sim.sleep(until=start_of_the_day+6*3600+30*60) # 6:30
        print("professor breakfasts at "+nowstr(sim))
        
        sim.sleep(until=start_of_the_day+6*3600+50*60) # 6:50
        print("professor showers at "+nowstr(sim))

        sim.sleep(until=start_of_the_day+7*3600+30*60) # 7:30
        print("professor leaves home and drives to school at "+nowstr(sim))

        if sim.now < 24*3600:
            # traffic jam at the first day
            sim.sleep(offset=2*3600+45*60) # 2 hours and 45 minutes from now
            print("professor arrives at school at "+nowstr(sim))

            if sim.now < 9*3600:
                # if arrives before the 9 o'clock, attend both meetings
                sim.sleep(until=9*3600)
                print("professor has first meeting at "+nowstr(sim))

                sim.sleep(until=10*3600)
                print("professor has second meeting at "+nowstr(sim))
            else:
                # if late, no the first meeting and resched the second
                sim.sleep(until=11*3600)
                print("professor has second meeting at "+nowstr(sim))
        else:
            # no traffic jam in the following days
            sim.sleep(offset=45*60) # 45 minutes from now
            print("professor arrives at school at "+nowstr(sim))

        # the rest of the day is a blur for the professor
        rest_of_the_day(sim, start_of_the_day)

def rest_of_the_day(sim, start):
    # sleep until the start of the next day
    sim.sleep(until=start+24*3600)
            
sim = simulus.simulator()
sim.process(prof_life) 
sim.run(until=72*3600)

professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 10:15:00
professor has second meeting at 11:00:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor sho

Rather than having many event handlers as in the earlier example, the professor's life now becomes one process. It's one big loop and one iteration of the loop represents one day of the professor's life. At each iteration, the professor calls `sleep()` to advance the time. 

There is one caveat for using the `sleep()` method: it can only be called within a process function, i.e., the starting function of a process, or a function directly or indirectly called by the starting function. Otherwise, simulus will raise an exception.

The professor goes through all the chores of the day. At the end of those chores, she calls the function `rest_of_the_day()` to simulate all the activities she takes for the rest of the day. We are still in the same process. the function sleeps until the beginning of the next day and returns. The loop continues to the next iteration for the next day. 

#### Processes Everywhere

Of course the world does not have just the professor. There are other professors, students, and many other people. If we simulate a multi-agent system, for example, each agent can be treated an automic entity with its own agenda. So it's natural to simulate such a system using processes to represent the independent agents. Each agent may also use multiple processes. Recall that professor in the previous example actually has coffee and reads at the same time. Conceptually the two overlapping activities can be modeled as separate processes.

Our world is a complicated world. And processes do interact. The agents in a multi-agent system may communicate and synchronize with one another and they compete for resources. Simulus provides the necessary facilities for processes to fulfill these functions. We will deal with the more complicated issues later.

For now, we examine a simple example showing the creation and execution of multiple processes, and a simple way to synchronize them (by waiting for the completion of the processes).

In [8]:
# examples/basics/homework.py
import simulus

from random import seed, expovariate, gauss
seed(12345)

def work(sim, params):
    student_id = params.get("student_id")
    print("student %d starts to work at %g" % 
          (student_id, sim.now))
    sim.sleep(gauss(30, 5)) # work on part one
    sim.sleep(expovariate(1/10.)) # take some rest
    sim.sleep(gauss(60, 10)) # work on part two
    print("student %d finishes working at %g" % 
          (student_id, sim.now))

def homework(sim, params):
    print("homework assigned at "+str(sim.now))
    # five students working on the homework each starts at a random 
    # time (exponentially distributed with mean of 10)
    students = []
    for i in range(5):
        s = sim.process(work, expovariate(1/10.), student_id=i)
        students.append(s)
    # wait for all student processes to complete
    sim.join(students)
    print("last student finishes homework at "+str(sim.now))
        
sim = simulus.simulator()
sim.process(homework, 10) 
sim.run()

homework assigned at 10
student 1 starts to work at 10.1022
student 3 starts to work at 13.5473
student 4 starts to work at 14.5952
student 0 starts to work at 15.3892
student 2 starts to work at 27.4415
student 4 finishes working at 108.348
student 1 finishes working at 109.514
student 2 finishes working at 111.156
student 0 finishes working at 122.177
student 3 finishes working at 141.183
last student finishes homework at 141.18344556219594
